# in this file we will check the avg voting method of models

In [5]:
"""
Soft-Voting Grid Search (10 models, combos 2–10) – FINAL
=======================================================

Outputs
-------
soft_vote_results.csv          – metrics for 1 013 soft-vote subsets
individual_model_metrics.csv   – metrics for each single model
"""
import itertools, sys, time
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    accuracy_score, confusion_matrix
)
from tqdm.auto import tqdm

# ──────────────────────────────────────────────────────────────
# CONFIG
# ──────────────────────────────────────────────────────────────
BASE = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
            r"\Stock-Market-Prediction\Final_runs_csv")

FILES = {
    "catboost" : "catboost_trial57_predictions.csv",
    "cnn"      : "cnn_predictions.csv",
    "gru"      : "gru_trial28_f05_preds.csv",
    "lgbm"     : "lgbm_predictions_formatted_backup.csv",
    "logreg"   : "logisticreg_validation_predictions.csv",
    "lstm"     : "lstm_test_predictions.csv",
    "rf"       : "RandomForest_predictions_custom_high_precision.csv",
    "tcn"      : "TCN_Trial_36_predictions.csv",
    "xgb"      : "xgboost_predictions_fixed.csv",
    "cnn_lstm" : "cnn_lstm_val_preds_20250614_142329.csv",
}

THRESH = 0.50   # avg_prob > THRESH → UP

# ──────────────────────────────────────────────────────────────
# 1. LOAD & ALIGN
# ──────────────────────────────────────────────────────────────
earliest = {}

def load_one(path: Path, key: str) -> pd.DataFrame:
    """
    Return dataframe with robust timestamp parse:
        timestamp | <key> (prob_up) | <key>_actual
    Tries both dayfirst=False and dayfirst=True, keeps the better parse.
    """
    df = pd.read_csv(path)
    need = {"timestamp", "prob_up", "actual"}
    miss = need - set(df.columns)
    if miss:
        sys.exit(f"❌ {path.name}: missing {miss}")

    raw = df["timestamp"]
    ts_mdy = pd.to_datetime(raw, dayfirst=False, utc=True, errors="coerce")
    ts_dmy = pd.to_datetime(raw, dayfirst=True,  utc=True, errors="coerce")

    if ts_mdy.notna().sum() == ts_dmy.notna().sum() == 0:
        sys.exit(f"❌ unparsable timestamps in {path.name}")

    ts = ts_dmy if ts_dmy.notna().sum() > ts_mdy.notna().sum() else ts_mdy
    earliest[key] = ts.min()

    return pd.DataFrame({
        "timestamp": ts,
        key: df["prob_up"].astype(np.float32),
        f"{key}_actual": df["actual"].astype(np.uint8)
    })

print("🔧 Loading prediction files …")
merged = None
for key, fname in FILES.items():
    fp = BASE / fname
    if not fp.exists():
        sys.exit(f"❌ missing {fp}")
    part = load_one(fp, key)
    merged = part if merged is None else merged.merge(part, on="timestamp", how="inner")

# trim to latest common start date
start_cut = max(earliest.values())
merged = merged[merged["timestamp"] >= start_cut].reset_index(drop=True)
print(f"Rows after inner-join & trim: {len(merged):,}")

# verify identical actuals
act_cols = [c for c in merged.columns if c.endswith("_actual")]
if not (merged[act_cols].nunique(axis=1) == 1).values.all():
    sys.exit("❌ 'actual' columns differ – fix before continuing")

y_true = merged[act_cols[0]].to_numpy(dtype=np.uint8)
merged.drop(columns=act_cols, inplace=True)

model_keys = list(FILES.keys())
prob_mat = merged[model_keys].to_numpy(dtype=np.float32)

# ──────────────────────────────────────────────────────────────
# 2. EXHAUSTIVE SOFT-VOTE SEARCH
# ──────────────────────────────────────────────────────────────
def avg_vote(mat: np.ndarray) -> np.ndarray:
    return (mat.mean(axis=1) > THRESH).astype(np.uint8)

n = len(model_keys)
total = (2**n) - 1 - n
results = []

t0 = time.time()
with tqdm(total=total, unit="combo") as bar:
    for k in range(2, n+1):
        for idx in itertools.combinations(range(n), k):
            y_pred = avg_vote(prob_mat[:, idx])

            prec = precision_score(y_true, y_pred, zero_division=0)
            rec  = recall_score   (y_true, y_pred, zero_division=0)
            f1   = f1_score       (y_true, y_pred, zero_division=0)
            acc  = accuracy_score(y_true, y_pred)
            tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

            results.append({
                "models"   : "+".join(model_keys[i] for i in idx),
                "n_models" : k,
                "precision": round(prec, 6),
                "recall"   : round(rec, 6),
                "f1"       : round(f1, 6),
                "accuracy" : round(acc, 6),
                "tp": tp, "fp": fp, "tn": tn, "fn": fn
            })
            bar.update(1)

print(f"🏁 completed {total} soft-vote combos in {time.time()-t0:.1f}s")

# ──────────────────────────────────────────────────────────────
# 3. SINGLE-MODEL METRICS
# ──────────────────────────────────────────────────────────────
indiv = []
for i, name in enumerate(model_keys):
    y_pred = (prob_mat[:, i] > THRESH).astype(np.uint8)
    indiv.append({
        "models": name, "n_models": 1,
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall"   : recall_score  (y_true, y_pred, zero_division=0),
        "f1"       : f1_score      (y_true, y_pred, zero_division=0),
        "accuracy" : accuracy_score(y_true, y_pred)
    })
indiv_df = pd.DataFrame(indiv).sort_values("f1", ascending=False)

# ──────────────────────────────────────────────────────────────
# 4. SAVE OUTPUTS
# ──────────────────────────────────────────────────────────────
out_soft = BASE / "soft_vote_results.csv"
out_ind  = BASE / "individual_model_metrics.csv"

(pd.DataFrame(results)
   .sort_values(["precision", "recall","f1"],

                ascending=[False, False, True])
   .to_csv(out_soft, index=False))
indiv_df.to_csv(out_ind, index=False)

# ──────────────────────────────────────────────────────────────
# 5. CONSOLE SUMMARY
# ──────────────────────────────────────────────────────────────
print("\nTop-10 soft ensembles by F1:")
top10 = pd.read_csv(out_soft).head(15)
print(top10[["models","n_models","precision","recall","f1"]].to_string(index=False))

best_single = indiv_df.iloc[0]
best_combo  = top10.iloc[0]
improv = best_combo["f1"] - best_single["f1"]

print("\nBest single model :",
      f"{best_single['models']}  F1={best_single['f1']:.4f}")
print("Best soft ensemble:",
      f"{best_combo['models']}  F1={best_combo['f1']:.4f}")
print(f"Improvement       : {improv:+.4f} "
      f"({improv/best_single['f1']*100:+.2f}%)")
print(f"\n💾 Files written: {out_soft.name}, {out_ind.name}")


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5968\3355244158.py:61: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  ts_dmy = pd.to_datetime(raw, dayfirst=True,  utc=True, errors="coerce")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5968\3355244158.py:61: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  ts_dmy = pd.to_datetime(raw, dayfirst=True,  utc=True, errors="coerce")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5968\3355244158.py:61: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  ts_dmy = pd.to_datetime(raw, dayfirst=True,  utc=True, errors="coerce")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5968\3355244158.py:60: UserWarning: Parsing dates in %d/%m/%Y %H:

🔧 Loading prediction files …
Rows after inner-join & trim: 3,105


100%|██████████| 1013/1013 [00:03<00:00, 273.78combo/s]

🏁 completed 1013 soft-vote combos in 3.7s

Top-10 soft ensembles by F1:
                    models  n_models  precision   recall       f1
                  gru+lgbm         2   0.637500 0.031481 0.060000
                gru+rf+tcn         3   0.631222 0.172222 0.270611
              gru+lgbm+tcn         3   0.630435 0.035802 0.067757
                    gru+rf         2   0.628770 0.167284 0.264261
     catboost+gru+lgbm+tcn         4   0.625000 0.098765 0.170576
    catboost+gru+lgbm+lstm         4   0.624464 0.179630 0.279003
catboost+gru+lgbm+lstm+tcn         5   0.623950 0.183333 0.283397
           catboost+gru+rf         3   0.619863 0.223457 0.328494
         catboost+gru+lgbm         3   0.619433 0.094444 0.163899
               gru+lgbm+rf         3   0.619342 0.185802 0.285850
             gru+lgbm+lstm         3   0.618384 0.137037 0.224356
                  gru+lstm         2   0.617544 0.108642 0.184777
       catboost+gru+rf+tcn         4   0.617450 0.227160 0.332130
    